In [66]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from collections import defaultdict
from surprise.model_selection import train_test_split
import io

from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from TfidfLevenshteinAlgorithm import TfidfLevenshteinAlgorithm

In [67]:
path = 'clustered_clients10000.csv'
df = pd.read_csv(path)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0.1               10000 non-null  int64  
 1   Unnamed: 0                 10000 non-null  int64  
 2   client_id                  10000 non-null  object 
 3   Cluster                    10000 non-null  int64  
 4   DistinctCountOrders        10000 non-null  int64  
 5   AverageOrderSum            10000 non-null  float64
 6   AvgBill                    10000 non-null  float64
 7   AverageNumItemsInOrder     10000 non-null  float64
 8   RevenueClient              10000 non-null  float64
 9   TotalNumberItemsOrdered    10000 non-null  int64  
 10  AverageMarginOrder         10000 non-null  float64
 11  cnt_purchased_orders       10000 non-null  int64  
 12  AvgOrderSumPurchased       10000 non-null  float64
 13  AvgNumItemsOrderPurchased  10000 non-null  floa

In [69]:
for col in df.columns:
    if type(df[col][0]) == np.float64:
        df[col] = df[col].astype('float16')
    elif type(df[col][0]) == np.int64:
        df[col] = df[col].astype('int16')
    elif type(df[col][0]) == np.object:
        df[col] = df[col].astype('category')
    else:
        pass

/var/folders/hz/np5gwl5d697cq7qxk89wms9c0000gp/T/ipykernel_17356/2894629639.py:6: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif type(df[col][0]) == np.object:


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0.1               10000 non-null  int16  
 1   Unnamed: 0                 10000 non-null  int16  
 2   client_id                  10000 non-null  object 
 3   Cluster                    10000 non-null  int16  
 4   DistinctCountOrders        10000 non-null  int16  
 5   AverageOrderSum            10000 non-null  float16
 6   AvgBill                    10000 non-null  float16
 7   AverageNumItemsInOrder     10000 non-null  float16
 8   RevenueClient              10000 non-null  float16
 9   TotalNumberItemsOrdered    10000 non-null  int16  
 10  AverageMarginOrder         10000 non-null  float16
 11  cnt_purchased_orders       10000 non-null  int16  
 12  AvgOrderSumPurchased       10000 non-null  float16
 13  AvgNumItemsOrderPurchased  10000 non-null  floa

In [71]:
df.head()

,Unnamed: 0.1,Unnamed: 0,client_id,Cluster,DistinctCountOrders,AverageOrderSum,AvgBill,AverageNumItemsInOrder,RevenueClient,TotalNumberItemsOrdered,...,КГТ,ОДЕЖДА,ППКП,CENTRAL,FAR EAST,NORTH,PRIVOLZIE,SIBERIA,SOUTHERN,URAL
0,0,0,55575053-54505550565678,2,100,1759.0,1723.0,2.439453,inf,244,...,0.192627,0.639160,0.024597,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,55575048-48524956565375,1,66,580.5,524.0,3.787109,34592.0,250,...,0.012001,0.320068,0.459961,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,2,55575555-57524953505673,3,54,823.0,807.0,2.703125,43584.0,146,...,0.000000,0.952148,0.006851,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,55574953-50505150484978,2,59,3608.0,3482.0,4.167969,inf,246,...,0.187012,0.569336,0.105713,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,55575054-51515151484875,2,46,2670.0,2632.0,1.717773,inf,79,...,0.455811,0.405029,0.088623,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
print(df.head())

   Unnamed: 0.1  Unnamed: 0                client_id  Cluster  \
0             0           0  55575053-54505550565678        2   
1             1           1  55575048-48524956565375        1   
2             2           2  55575555-57524953505673        3   
3             3           3  55574953-50505150484978        2   
4             4           4  55575054-51515151484875        2   

   DistinctCountOrders  AverageOrderSum  AvgBill  AverageNumItemsInOrder  \
0                  100           1759.0   1723.0                2.439453   
1                   66            580.5    524.0                3.787109   
2                   54            823.0    807.0                2.703125   
3                   59           3608.0   3482.0                4.167969   
4                   46           2670.0   2632.0                1.717773   

   RevenueClient  TotalNumberItemsOrdered  ...       КГТ    ОДЕЖДА      ППКП  \
0            inf                      244  ...  0.192627  0.639160  0.02

In [73]:
print(df.dtypes)

Unnamed: 0.1                   int16
Unnamed: 0                     int16
client_id                     object
Cluster                        int16
DistinctCountOrders            int16
AverageOrderSum              float16
AvgBill                      float16
AverageNumItemsInOrder       float16
RevenueClient                float16
TotalNumberItemsOrdered        int16
AverageMarginOrder           float16
cnt_purchased_orders           int16
AvgOrderSumPurchased         float16
AvgNumItemsOrderPurchased    float16
RevenueClientPurchased       float16
PurchasedItems               float16
AvgMarginPurchased           float16
SharePurchasedOrders         float16
ДЕТСКОЕ ПИТАНИЕ              float16
ЖЕНСКИЕ ШТУЧКИ               float16
ИГРУШКИ                      float16
КАНЦТОВАРЫ, КНИГИ, ДИСКИ     float16
КОСМЕТИКА/ГИГИЕНА            float16
КРУПНОГАБАРИТНЫЙ ТОВАР       float16
ОБУВЬ                        float16
ПОДГУЗНИКИ                   float16
СОПУТСТВУЮЩИЕ ТОВАРЫ         float16
Т

In [74]:
choose_column = ['AverageOrderSum']
selected_columns = ['client_id', 'Cluster'] + choose_column

In [75]:
selected_columns

['client_id', 'Cluster', 'AverageOrderSum']

In [76]:
maxi = df[choose_column].max().max()
mini = df[choose_column].min().min()

reader = Reader(rating_scale=(mini, maxi))

In [77]:
# df[selected_columns] = df[selected_columns][:10]

In [78]:
df[selected_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   client_id        10000 non-null  object 
 1   Cluster          10000 non-null  int16  
 2   AverageOrderSum  10000 non-null  float16
dtypes: float16(1), int16(1), object(1)
memory usage: 117.3+ KB


In [79]:
data = Dataset.load_from_df(df[selected_columns], reader)

In [80]:
data

In [81]:
trainset, testset = train_test_split(data, test_size=.25)

sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)

In [82]:
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [83]:
predictions = algo.test(testset)

In [84]:
#predictions to dataframe
predictions_df = pd.DataFrame(predictions)

In [85]:
print(predictions_df)

                          uid  iid    r_ui          est  \
0     55575448-49554954555571    0   835.0  2073.815326   
1     55575057-54565248565679    0  1175.0  2073.815326   
2     55575055-54555552505375    0  1648.0  2073.815326   
3     55574856-51495449505572    0  2480.0  2073.815326   
4     55575655-57535251485574    0  1248.0  2073.815326   
...                       ...  ...     ...          ...   
2495  55574953-50535053565777    0  3320.0  2073.815326   
2496  55575048-54494856555576    0  2424.0  2073.815326   
2497  55574852-57555548554872    0  3118.0  2073.815326   
2498  55575050-53524956575674    0   166.0  2073.815326   
2499  55575155-50484849484872    0  4972.0  2073.815326   

                        details  
0     {'was_impossible': False}  
1     {'was_impossible': False}  
2     {'was_impossible': False}  
3     {'was_impossible': False}  
4     {'was_impossible': False}  
...                         ...  
2495  {'was_impossible': False}  
2496  {'was_impossi

In [86]:
#show similar users
algo.get_neighbors(1, k=10)

[0, 2, 3]

In [87]:
#show unique users
print(df['client_id'].unique())

['55575053-54505550565678' '55575048-48524956565375'
 '55575555-57524953505673' ... '55575656-53535254535671'
 '55575656-53535053504870' '55575656-53534953495678']


In [88]:
for i in range(10):
    try:
        client = df['client_id'].unique()[i]
        print("client: ", client, "cluster: ", df[df['client_id'] == client]['Cluster'].values[0], "prediction: ",algo.get_neighbors(i, k=10))
    except Exception as e:
        print(e)
        pass


client:  55575053-54505550565678 cluster:  2 prediction:  [1, 2, 3]
client:  55575048-48524956565375 cluster:  1 prediction:  [0, 2, 3]
client:  55575555-57524953505673 cluster:  3 prediction:  [0, 1, 3]
client:  55574953-50505150484978 cluster:  2 prediction:  [0, 1, 2]
index 4 is out of bounds for axis 0 with size 4
index 5 is out of bounds for axis 0 with size 4
index 6 is out of bounds for axis 0 with size 4
index 7 is out of bounds for axis 0 with size 4
index 8 is out of bounds for axis 0 with size 4
index 9 is out of bounds for axis 0 with size 4


In [89]:
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

In [90]:
# # Create an instance of the TfidfLevenshteinAlgorithm
algo_Tfidf = TfidfLevenshteinAlgorithm(sim_options={'name': 'cosine', 'user_based': False, 'k': 10})

In [91]:
benchmark = []
# Iterate over all algorithms

algorithms = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering(), algo_Tfidf]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)

    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')


Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x105fdae60>, <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x138c7fc70>, <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x138c7fee0>, <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x12735d2a0>, <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x12735fe20>, <surprise.prediction_algorithms.knns.KNNBaseline object at 0x12735eb00>, <surprise.prediction_algorithms.knns.KNNBasic object at 0x12735e500>, <surprise.prediction_algorithms.knns.KNNWithMeans object at 0x12735d0c0>, <surprise.prediction_algorithms.knns.KNNWithZScore object at 0x12735c9a0>, <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x12735eb60>, <surprise.prediction_algorithms.co_clustering.CoClustering object at 0x12735da20>, <TfidfLevenshteinAlgorithm.TfidfLevenshteinAlgorithm object at 0x138cd5510>] 



Starting:  <surprise.prediction_algorithms.

/var/folders/hz/np5gwl5d697cq7qxk89wms9c0000gp/T/ipykernel_17356/881450337.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Done:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x105fdae60> 


Starting:  <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x138c7fc70>


/var/folders/hz/np5gwl5d697cq7qxk89wms9c0000gp/T/ipykernel_17356/881450337.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Done:  <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x138c7fc70> 


Starting:  <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x138c7fee0>


...

/var/folders/hz/np5gwl5d697cq7qxk89wms9c0000gp/T/ipykernel_17356/881450337.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Estimating biases using als...
Estimating biases using als...
Done:  <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x12735eb60> 


Starting:  <surprise.prediction_algorithms.co_clustering.CoClustering object at 0x12735da20>


/var/folders/hz/np5gwl5d697cq7qxk89wms9c0000gp/T/ipykernel_17356/881450337.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
/var/folders/hz/np5gwl5d697cq7qxk89wms9c0000gp/T/ipykernel_17356/881450337.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Done:  <surprise.prediction_algorithms.co_clustering.CoClustering object at 0x12735da20> 


Starting:  <TfidfLevenshteinAlgorithm.TfidfLevenshteinAlgorithm object at 0x138cd5510>
Done:  <TfidfLevenshteinAlgorithm.TfidfLevenshteinAlgorithm object at 0x138cd5510> 



	DONE



/var/folders/hz/np5gwl5d697cq7qxk89wms9c0000gp/T/ipykernel_17356/881450337.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


In [92]:
print(pd.DataFrame(benchmark).set_index('Algorithm').sort_values('fit_time'))

                              test_rmse  fit_time  test_time
Algorithm                                                   
TfidfLevenshteinAlgorithm   2301.005882  0.004523   0.018390
NormalPredictor             3117.565557  0.006013   0.014326
BaselineOnly                1816.274099  0.027506   0.009038
SlopeOne                    2298.567805  0.095526   0.011396
SVDpp                      43334.578727  0.103163   0.011571
SVD                        43334.562524  0.170590   0.013935
NMF                         2297.943759  0.447778   0.018099
CoClustering                2302.702863  0.813285   0.008845
KNNBasic                    2300.780109  2.165938   0.014785
KNNBaseline                 1817.084553  2.271427   0.012248
KNNWithMeans                2293.997759  2.422793   0.012992
KNNWithZScore               2289.266913  3.108731   0.014852
